In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
#import kelp_param
import kelp3d_objs as k3
import ipyparallel as ipp
import concurrent.futures as cf
import itertools as it
#from sympy import divisors
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as opt
from scipy.interpolate import interp1d
import ipyvolume as ipv
import ipywidgets as ipw

import kelp_compute
import kelp_analyze

In [8]:
import sqlite3
import shutil
import os

### Define Constants

In [9]:
out = ipw.Output()

In [10]:
out

Output()

In [11]:
# Hard-coded parameters
rope_spacing = 20.0 # (m)
zmin, zmax = 0.0, 10.0 # (m)

# Light from above
I0 = 50.0 # (W/m^2)
phi_s = 0 # (rad)
theta_s = 0 # (rad)
decay = 0 # (?) - light (practically) only from directly above

# Calculated or otherwise boring parameters
xmin = -rope_spacing/2
xmax = rope_spacing/2
ymin = -rope_spacing/2
ymax = rope_spacing/2

# Set parameters
study_name = 'single'
a_water = 0.179
b = 0.219
kelp_dist='top-heavy'

data_dir = '../../results'

ns = 14
nz = 14
na = 18

num_scatters = 1
fd_flag = False
lis_options = "-i gmres -restart 10000 -tol 1e-4"

# Compute

shutil.rmtree(os.path.join(data_dir, study_name), ignore_errors=True)

base_dir, db_path, table_name = kelp_compute.create_table(study_name, data_dir)

with out:
    kelp_compute.kelp_calculate(
        base_dir, db_path, table_name,
        a_water, b,
        ns, nz, na,
        kelp_dist, num_scatters,
        fd_flag,
        lis_options
    )

# Retrieve

conn = sqlite3.connect(db_path)
res = kelp_analyze.query_results(conn, table_name, ns=ns, nz=nz, na=na)[0]
conn.close()
irrad = res['irrad'][:]

print("{:.2f}% nan".format(np.sum(np.isnan(irrad))/np.size(irrad)))

# Visualize

ipv.figure()
ipv.volshow(irrad)
ipv.show()

0.36% nan


/home/oliver/local/miniconda3/envs/kelp/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/oliver/local/miniconda3/envs/kelp/lib/python3.6/site-packages/ipyvolume/serialize.py:62: RuntimeWarning: invalid value encountered in true_divide
  grid_normalized = (grid*1.0 - vmin) / (vmax - vmin)
/home/oliver/local/miniconda3/envs/kelp/lib/python3.6/site-packages/ipyvolume/serialize.py:66: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)
